In [ ]:
import os
import cv2
import numpy as np

import glob
from keras._tf_keras.keras.preprocessing.image import img_to_array
from keras._tf_keras.keras.preprocessing.image import ImageDataGenerator
from mtcnn import MTCNN
from keras._tf_keras.keras.models import Sequential
from keras._tf_keras.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [ ]:
# Load OpenCV's Haar Cascade Face Detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")


# Load MTCNN Face Detector
mtcnn_detector = MTCNN()

# Image Size
IMG_SIZE = (200, 200)


# Dataset Paths
folder_path = "D:/fold/Project/im/dataset/combined_faces/content/data"


In [17]:
def load_and_preprocess(img_path):
    img = cv2.imread(img_path)
    if img is None:
        print(f"Error loading image: {img_path}")
        return None
    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, IMG_SIZE)
    img = img.astype("float32") / 255.0
    return img_to_array(img)

In [18]:
def detect_and_crop_face(img_path):
    img = cv2.imread(img_path)
    if img is None:
        print(f"Error: Unable to read {img_path}")
        return None

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Try Haar Cascade
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.02, minNeighbors=4, minSize=(30, 30))

    
    if len(faces) == 0:
        # Try MTCNN
        mtcnn_faces = mtcnn_detector.detect_faces(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        if len(mtcnn_faces) > 0:
            x, y, w, h = mtcnn_faces[0]['box']
        else:
            print(f"No face detected in {img_path}")
            return None
    else:
        x, y, w, h = faces[0]
    
    # Crop and resize face
    face = img[y:y+h, x:x+w]
    face = cv2.resize(face, IMG_SIZE)
    return face.astype("float32") / 255.0

In [19]:
# Load Images from Folder

# Load Images from Folder
def load_img_folder(folder_path):
    img_data = []
    if not os.path.exists(folder_path):
        raise FileNotFoundError(f"Folder not found: {folder_path}")
    
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith((".jpg", ".png", ".jpeg"))]
    if not image_files:
        print("No images found in dataset folder!")
        return np.array([])
    
    for filename in image_files:
        img_path = os.path.join(folder_path, filename)
        img_array = load_and_preprocess(img_path)
        if img_array is not None:
            img_data.append(img_array)
    
    return np.array(img_data)

In [20]:
if not os.path.exists(h5_file):
    with h5py.File(h5_file, "w") as hf:
        img_data, age_labels = [], []
        
        for img_path in glob.glob(os.path.join(folder_path, "*.jpg")):
            try:
                filename = os.path.basename(img_path)
                age = int(filename.split("_")[0])  # Extract age from filename
                img = detect_and_crop_face(img_path)
                
                if img is not None:
                    img_data.append(img)
                    age_labels.append(age)
            except Exception as e:
                print(f" Skipping {img_path}: {e}")

        if img_data:
            hf.create_dataset("images", data=np.array(img_data, dtype="float32"))
            hf.create_dataset("ages", data=np.array(age_labels, dtype="float32"))
            print(f"Dataset stored in {h5_file}")
        else:
            print(" No valid data to store in dataset!")

In [21]:
# Load Processed Dataset
X_train, y_train = [], []
image_paths = glob.glob(os.path.join(folder_path, "*.jpg"))

for img_path in image_paths:
    try:
        filename = os.path.basename(img_path)
        age = int(filename.split("_")[0])
        img = load_and_preprocess(img_path)
        if img is not None:
            X_train.append(img)
            y_train.append(age)
    except Exception as e:
        print(f" Skipping {img_path}: {e}")

X_train = np.array(X_train, dtype="float32")
y_train = np.array(y_train, dtype="float32")
print(f" Dataset Loaded: {X_train.shape[0]} images")


Dataset Loaded: 33486 images


In [22]:
# Model Architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='linear')  # Regression output for age
])

d:\fold\Project\im\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [23]:
# Compile and Train Model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 198, 198, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 99, 99, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 97, 97, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 48, 48, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 46, 46, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 23, 23, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 67712)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │    17,334,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,428,033 (66.48 MB)

 Trainable params: 17,428,033 (66.48 MB)

 Non-trainable params: 0 (0.00 B)

In [24]:
model.fit(X_train, y_train, batch_size=32, epochs=20, validation_split=0.2)

Epoch 1/20
838/838 ━━━━━━━━━━━━━━━━━━━━ 235s 274ms/step - loss: 231.1660 - mae: 11.4948 - val_loss: 762.9248 - val_mae: 24.1438
Epoch 2/20
838/838 ━━━━━━━━━━━━━━━━━━━━ 207s 247ms/step - loss: 99.4252 - mae: 7.4963 - val_loss: 789.6681 - val_mae: 24.1488
Epoch 3/20
838/838 ━━━━━━━━━━━━━━━━━━━━ 205s 244ms/step - loss: 82.6357 - mae: 6.8057 - val_loss: 582.3031 - val_mae: 20.1488
Epoch 4/20
838/838 ━━━━━━━━━━━━━━━━━━━━ 204s 244ms/step - loss: 72.9073 - mae: 6.4022 - val_loss: 541.7045 - val_mae: 19.4048
Epoch 5/20
838/838 ━━━━━━━━━━━━━━━━━━━━ 201s 240ms/step - loss: 61.7369 - mae: 5.9021 - val_loss: 554.6614 - val_mae: 19.5674
Epoch 6/20
838/838 ━━━━━━━━━━━━━━━━━━━━ 214s 256ms/step - loss: 57.6120 - mae: 5.6754 - val_loss: 518.7498 - val_mae: 18.7830
Epoch 7/20
838/838 ━━━━━━━━━━━━━━━━━━━━ 218s 260ms/step - loss: 52.5935 - mae: 5.3876 - val_loss: 347.1629 - val_mae: 15.0328
Epoch 8/20
838/838 ━━━━━━━━━━━━━━━━━━━━ 200s 239ms/step - loss: 45.5868 - mae: 5.0047 - val_loss: 398.3991 - val_mae

In [25]:
# Save Model
model.save("age_detection_model.h5")
print("Model saved successfully!")

 Model saved successfully!
